In [1]:
%reset -fs
import bs4
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import requests
import pickle
import re

In [2]:
df = pd.read_pickle('df_all_movies_2017_pk')
df

,Rank,Movie,ReleaseDate,Distributor,Genre,Revenues,Tickets_Sold,summary_page_url,cast_page_url
0,1,Star Wars Ep. VIII: The Las…,"Dec 15, 2017",Walt Disney,Adventure,"$517,218,368","57,660,910",https://www.the-numbers.com/movie/Star-Wars-Ep...,https://www.the-numbers.com/movie/Star-Wars-Ep...
1,2,Beauty and the Beast,"Mar 17, 2017",Walt Disney,Musical,"$504,014,165","56,188,870",https://www.the-numbers.com/movie/Beauty-and-t...,https://www.the-numbers.com/movie/Beauty-and-t...
2,3,Wonder Woman,"Jun 2, 2017",Warner Bros.,Action,"$412,563,408","45,993,691",https://www.the-numbers.com/movie/Wonder-Woman...,https://www.the-numbers.com/movie/Wonder-Woman...
3,4,Guardians of the Galaxy Vol 2,"May 5, 2017",Walt Disney,Action,"$389,813,101","43,457,424",https://www.the-numbers.com/movie/Guardians-of...,https://www.the-numbers.com/movie/Guardians-of...
4,5,Spider-Man: Homecoming,"Jul 7, 2017",Sony Pictures,Action,"$334,201,140","37,257,652",https://www.the-numbers.com/movie/Spider-Man-H...,https://www.the-numbers.com/movie/Spider-Man-H...
...,...,...,...,...,...,...,...,...,...
729,730,Mifune,"Feb 25, 2000",Sony Pictures Cla…,Drama,$398,44,https://www.the-numbers.com/movie/Mifune#tab=s...,https://www.the-numbers.com/movie/Mifune#tab=c...
730,731,Gridiron Heroes,"Apr 15, 2016",Indican Pictures,Documentary,$250,27,https://www.the-numbers.com/movie/Gridiron-Her...,https://www.the-numbers.com/movie/Gridiron-Her...
731,732,National Bird,"Nov 11, 2016",FilmRise,Documentary,$186,20,https://www.the-numbers.com/movie/National-Bir...,https://www.the-numbers.com/movie/National-Bir...
732,733,Nightlights,"Aug 15, 2014",Indican Pictures,Drama,$159,17,https://www.the-numbers.com/movie/Nightlights-...,https://www.the-numbers.com/movie/Nightlights-...


In [3]:
row_count = df.shape[0]
row_count

734

In [4]:
i = 0
r = requests.get(df.loc[i,'summary_page_url']) #location in my dataframe
s = bs(r.text)

ex_all_tables = s.find_all('table')
ex_row = ex_all_tables[2].find_all('tr')
ex_row

[<tr><td><b>Opening Weekend:</b></td><td>$220,009,584 (35.5% of total gross)</td></tr>,
 <tr><td><b>Legs:</b></td><td>2.82 (domestic box office/biggest weekend)</td></tr>,
 <tr><td><b>Domestic Share:</b></td><td>46.6% (domestic box office/worldwide)</td></tr>,
 <tr><td><b>Production Budget:</b></td><td>$262,000,000 (worldwide box office is 5.1 times production budget)</td></tr>,
 <tr><td><b>Theater counts:</b></td><td>4,232 opening theaters/4,232 max. theaters, 6.7 weeks average run per theater</td></tr>,
 <tr>
 <td><b>Infl. Adj. Dom. BO</b></td>
 <td>$631,702,051</td>
 </tr>]

In [5]:
# Extract legs trial
ex_table_metrics = ex_all_tables[2]
all_rows_metrics = ex_table_metrics.find_all('tr')
for row in all_rows_metrics:
    found_legs = False
    if ('Legs' in row.text):
        ex_rows_legs = row
        found_legs = True
        break

if (found_legs):
    ex_strg_legs = ex_rows_legs.text
    print(ex_strg_legs)

    pattern = re.compile(r"[-+]?\d*\.\d+|\d+")
    print(pattern.findall(ex_strg_legs))
    legs = float(pattern.findall(ex_strg_legs)[0])
else:
    legs = 'not_found'
        
legs

Legs:2.82 (domestic box office/biggest weekend)
['2.82']


2.82

In [6]:
# Test Extract production budget 

ex_row_budget = ex_table_metrics.find_all('tr')[3]
ex_strg_budget = ex_row_budget.text
print(ex_strg_budget)

pattern = re.compile(r"[-+]?(?:\d*\.\d+|\d+)")

print(pattern.findall(ex_strg_budget))
budget_ratio_list = pattern.findall(ex_strg_budget)


ratio = float(budget_ratio_list.pop())
print(ratio)

budget_str = ''
budget_str = budget_str.join(budget_ratio_list)
print(budget_str)

budget = int(budget_str)
budget

Production Budget:$262,000,000 (worldwide box office is 5.1 times production budget)
['262', '000', '000', '5.1']
5.1
262000000


262000000

In [7]:
#extract theater count trial

ex_row_theater_count = ex_table_metrics.find_all('tr')[4]
ex_strg_theater_count = ex_row_theater_count.find_all('td')[1].text

pattern = re.compile(r"[-+]?(?:\d*\.\d+|\d+)")
print(pattern.findall(ex_strg_theater_count))
theater_count_list = pattern.findall(ex_strg_theater_count)

theater_count = int(theater_count_list[0] + theater_count_list[1])
print(theater_count)

theater_run = float(theater_count_list[-1])
print(theater_run)

['4', '232', '4', '232', '6.7']
4232
6.7


In [8]:
#Extract Movie Details table
ex_table_movie_details = ex_all_tables[5]

In [9]:
#Extract franchise trial

ex_row_franchise = ex_table_movie_details.find_all('tr')[5]
ex_elems = ex_row_franchise.find_all('td')[1].find_all('a')
for i in ex_elems:
    print(i.text)

Star Wars


In [10]:
#Extract source trial
ex_row_source = ex_table_movie_details.find_all('tr')[8]
source = ex_row_source.find_all('td')[1].find_all('a')[0].text
source

'Original Screenplay'

In [11]:
#Extract Genre trial
ex_row_genre = ex_table_movie_details.find_all('tr')[9]
genre = ex_row_genre.find_all('td')[1].find_all('a')[0].text
genre

'Adventure'

In [12]:
# Extract creative type trial
ex_row_ctype = ex_table_movie_details.find_all('tr')[11]
ctype = ex_row_ctype.find_all('td')[1].find_all('a')[0].text
ctype

'Science Fiction'

In [13]:
# Extract legs for all 2017 movies  v2

all_movies_legs_list = []
all_movies_all_tables_2017_list = []

for i in range(row_count):
    r = requests.get(df.loc[i,'summary_page_url']) #location in my dataframe
    s = bs(r.text)
    ex_all_tables = s.find_all('table')
    
    for table in ex_all_tables:
        all_rows = table.find_all('tr')
        found_legs = False
    
        for row in all_rows:
            if ('Legs' in row.text):
                ex_rows_legs = row
                found_legs = True

                ex_strg_legs = ex_rows_legs.text
                print(ex_strg_legs)

                pattern = re.compile(r"[-+]?\d*\.\d+|\d+")
                print(pattern.findall(ex_strg_legs))
                legs = float(pattern.findall(ex_strg_legs)[0])
                all_movies_legs_list.append(legs)
                break        
        if (found_legs):
            break
                
    if (not found_legs):
        legs = 1
        all_movies_legs_list.append(legs)
    
    print(legs)
    

legs_2017_df = pd.DataFrame(all_movies_legs_list, columns = ['Legs'])
legs_2017_df.to_pickle('legs_2017_df_pk')



Legs:2.82 (domestic box office/biggest weekend)
['2.82']
2.82
Legs:2.88 (domestic box office/biggest weekend)
['2.88']
2.88
Legs:4.00 (domestic box office/biggest weekend)
['4.00']
4.0
Legs:2.66 (domestic box office/biggest weekend)
['2.66']
2.66
Legs:2.86 (domestic box office/biggest weekend)
['2.86']
2.86
Legs:2.66 (domestic box office/biggest weekend)
['2.66']
2.66
Legs:2.57 (domestic box office/biggest weekend)
['2.57']
2.57
Legs:3.65 (domestic box office/biggest weekend)
['3.65']
3.65
Legs:2.56 (domestic box office/biggest weekend)
['2.56']
2.56
Legs:2.29 (domestic box office/biggest weekend)
['2.29']
2.29
Legs:2.44 (domestic box office/biggest weekend)
['2.44']
2.44
Legs:3.76 (domestic box office/biggest weekend)
['3.76']
3.76
Legs:4.14 (domestic box office/biggest weekend)
['4.14']
4.14
Legs:3.32 (domestic box office/biggest weekend)
['3.32']
3.32
Legs:5.27 (domestic box office/biggest weekend)
['5.27']
5.27
Legs:3.49 (domestic box office/biggest weekend)
['3.49']
3.49
Legs:2.74

Legs:3.15 (domestic box office/biggest weekend)
['3.15']
3.15
Legs:3.34 (domestic box office/biggest weekend)
['3.34']
3.34
Legs:2.03 (domestic box office/biggest weekend)
['2.03']
2.03
Legs:2.46 (domestic box office/biggest weekend)
['2.46']
2.46
Legs:2.48 (domestic box office/biggest weekend)
['2.48']
2.48
Legs:3.35 (domestic box office/biggest weekend)
['3.35']
3.35
Legs:2.54 (domestic box office/biggest weekend)
['2.54']
2.54
Legs:4.04 (domestic box office/biggest weekend)
['4.04']
4.04
Legs:4.18 (domestic box office/biggest weekend)
['4.18']
4.18
Legs:2.01 (domestic box office/biggest weekend)
['2.01']
2.01
Legs:1.86 (domestic box office/biggest weekend)
['1.86']
1.86
Legs:2.09 (domestic box office/biggest weekend)
['2.09']
2.09
Legs:8.98 (domestic box office/biggest weekend)
['8.98']
8.98
Legs:2.08 (domestic box office/biggest weekend)
['2.08']
2.08
Legs:4.18 (domestic box office/biggest weekend)
['4.18']
4.18
Legs:7.04 (domestic box office/biggest weekend)
['7.04']
7.04
Legs:4.0

Legs:3.24 (domestic box office/biggest weekend)
['3.24']
3.24
Legs:14.54 (domestic box office/biggest weekend)
['14.54']
14.54
Legs:4.31 (domestic box office/biggest weekend)
['4.31']
4.31
Legs:2.45 (domestic box office/biggest weekend)
['2.45']
2.45
Legs:69.07 (domestic box office/biggest weekend)
['69.07']
69.07
Legs:1.00 (domestic box office/biggest weekend)
['1.00']
1.0
Legs:5.53 (domestic box office/biggest weekend)
['5.53']
5.53
Legs:15.39 (domestic box office/biggest weekend)
['15.39']
15.39
Legs:18.60 (domestic box office/biggest weekend)
['18.60']
18.6
Legs:8.61 (domestic box office/biggest weekend)
['8.61']
8.61
Legs:2.12 (domestic box office/biggest weekend)
['2.12']
2.12
Legs:1.94 (domestic box office/biggest weekend)
['1.94']
1.94
Legs:6.89 (domestic box office/biggest weekend)
['6.89']
6.89
Legs:3.45 (domestic box office/biggest weekend)
['3.45']
3.45
Legs:13.05 (domestic box office/biggest weekend)
['13.05']
13.05
Legs:7.23 (domestic box office/biggest weekend)
['7.23']


Legs:2.57 (domestic box office/biggest weekend)
['2.57']
2.57
Legs:2.31 (domestic box office/biggest weekend)
['2.31']
2.31
Legs:2.14 (domestic box office/biggest weekend)
['2.14']
2.14
Legs:8.57 (domestic box office/biggest weekend)
['8.57']
8.57
Legs:3.66 (domestic box office/biggest weekend)
['3.66']
3.66
Legs:7.94 (domestic box office/biggest weekend)
['7.94']
7.94
Legs:5.74 (domestic box office/biggest weekend)
['5.74']
5.74
Legs:16.80 (domestic box office/biggest weekend)
['16.80']
16.8
Legs:2.20 (domestic box office/biggest weekend)
['2.20']
2.2
Legs:4.93 (domestic box office/biggest weekend)
['4.93']
4.93
Legs:2.01 (domestic box office/biggest weekend)
['2.01']
2.01
Legs:4.62 (domestic box office/biggest weekend)
['4.62']
4.62
Legs:6.82 (domestic box office/biggest weekend)
['6.82']
6.82
Legs:11.17 (domestic box office/biggest weekend)
['11.17']
11.17
Legs:1.93 (domestic box office/biggest weekend)
['1.93']
1.93
Legs:6.74 (domestic box office/biggest weekend)
['6.74']
6.74
Legs

Legs:1.00 (domestic box office/biggest weekend)
['1.00']
1.0
Legs:4.76 (domestic box office/biggest weekend)
['4.76']
4.76
Legs:10.99 (domestic box office/biggest weekend)
['10.99']
10.99
Legs:3.37 (domestic box office/biggest weekend)
['3.37']
3.37
Legs:2.85 (domestic box office/biggest weekend)
['2.85']
2.85
Legs:7.47 (domestic box office/biggest weekend)
['7.47']
7.47
1
Legs:5.22 (domestic box office/biggest weekend)
['5.22']
5.22
Legs:8.52 (domestic box office/biggest weekend)
['8.52']
8.52
Legs:4.04 (domestic box office/biggest weekend)
['4.04']
4.04
Legs:5.75 (domestic box office/biggest weekend)
['5.75']
5.75
Legs:4.57 (domestic box office/biggest weekend)
['4.57']
4.57
Legs:1.65 (domestic box office/biggest weekend)
['1.65']
1.65
Legs:1.92 (domestic box office/biggest weekend)
['1.92']
1.92
Legs:1.62 (domestic box office/biggest weekend)
['1.62']
1.62
Legs:5.28 (domestic box office/biggest weekend)
['5.28']
5.28
Legs:1.36 (domestic box office/biggest weekend)
['1.36']
1.36
Legs

1
Legs:1.00 (domestic box office/biggest weekend)
['1.00']
1.0
Legs:7.48 (domestic box office/biggest weekend)
['7.48']
7.48
Legs:1.94 (domestic box office/biggest weekend)
['1.94']
1.94
Legs:2.71 (domestic box office/biggest weekend)
['2.71']
2.71
Legs:1.00 (domestic box office/biggest weekend)
['1.00']
1.0
Legs:3.05 (domestic box office/biggest weekend)
['3.05']
3.05
Legs:3.57 (domestic box office/biggest weekend)
['3.57']
3.57
Legs:2.58 (domestic box office/biggest weekend)
['2.58']
2.58
Legs:1.70 (domestic box office/biggest weekend)
['1.70']
1.7
Legs:3.60 (domestic box office/biggest weekend)
['3.60']
3.6
Legs:1.00 (domestic box office/biggest weekend)
['1.00']
1.0
Legs:3.98 (domestic box office/biggest weekend)
['3.98']
3.98
Legs:1.00 (domestic box office/biggest weekend)
['1.00']
1.0
Legs:6.63 (domestic box office/biggest weekend)
['6.63']
6.63
Legs:19.00 (domestic box office/biggest weekend)
['19.00']
19.0
Legs:2.74 (domestic box office/biggest weekend)
['2.74']
2.74
Legs:1.19 

In [15]:
legs_2017_df

,Legs
0,2.82
1,2.88
2,4.00
3,2.66
4,2.86
...,...
729,11.47
730,16.78
731,3.52
732,3.03


In [16]:
# Extract budget and ratio for all  movies v2

all_movies_budget_list = []
all_movies_wbo_to_budget_ratio_list = []


for i in range(row_count):
    r = requests.get(df.loc[i,'summary_page_url']) #location in my dataframe
    s = bs(r.text)
    ex_all_tables = s.find_all('table')
    
    for table in ex_all_tables:
        all_rows = table.find_all('tr')
        found_budget = False
    
        for row in all_rows:
            if ('Budget' in row.text):
    #             print(row.text)
                found_budget = True
                ex_row_budget = row
                ex_strg_budget = ex_row_budget.text
    #             print(ex_strg_budget)

                pattern = re.compile(r"[-+]?(?:\d*\.\d+|\d+)")

    #             print(pattern.findall(ex_strg_budget))
                budget_ratio_list = pattern.findall(ex_strg_budget)


                ratio = float(budget_ratio_list.pop())
                print(ratio)

                budget_str = ''
                budget_str = budget_str.join(budget_ratio_list)
                print(budget_str)

                budget = int(budget_str)

                all_movies_budget_list.append(budget)
                all_movies_wbo_to_budget_ratio_list.append(ratio)
                break
        if(found_budget):
            break

    if (not found_budget):
        budget = 'nf'
        ratio = 'nf'
        all_movies_budget_list.append(budget)
        all_movies_wbo_to_budget_ratio_list.append(ratio)
    
    
    print(all_movies_budget_list)
    print(all_movies_wbo_to_budget_ratio_list)
    
    
budget_2017_df = pd.DataFrame(all_movies_budget_list, 
                                  columns = ['Production_Budget'])
wbo_to_budget_ratio_2017_df = pd.DataFrame(all_movies_wbo_to_budget_ratio_list,
                                               columns = ['WBO to Budget Ratio'])
     
budget_2017_df.to_pickle('budget_2017_pk')
wbo_to_budget_ratio_2017_df.to_pickle('wbo_to_budget_ratio_2017_pk')

5.1
262000000
[262000000]
[5.1]
8.0
160000000
[262000000, 160000000]
[5.1, 8.0]
5.5
150000000
[262000000, 160000000, 150000000]
[5.1, 8.0, 5.5]
4.3
200000000
[262000000, 160000000, 150000000, 200000000]
[5.1, 8.0, 5.5, 4.3]
5.0
175000000
[262000000, 160000000, 150000000, 200000000, 175000000]
[5.1, 8.0, 5.5, 4.3, 5.0]
20.0
35000000
[262000000, 160000000, 150000000, 200000000, 175000000, 35000000]
[5.1, 8.0, 5.5, 4.3, 5.0, 20.0]
4.7
180000000
[262000000, 160000000, 150000000, 200000000, 175000000, 35000000, 180000000]
[5.1, 8.0, 5.5, 4.3, 5.0, 20.0, 4.7]
13.8
75000000
[262000000, 160000000, 150000000, 200000000, 175000000, 35000000, 180000000, 75000000]
[5.1, 8.0, 5.5, 4.3, 5.0, 20.0, 4.7, 13.8]
4.8
127000000
[262000000, 160000000, 150000000, 200000000, 175000000, 35000000, 180000000, 75000000, 127000000]
[5.1, 8.0, 5.5, 4.3, 5.0, 20.0, 4.7, 13.8, 4.8]
4.9
250000000
[262000000, 160000000, 150000000, 200000000, 175000000, 35000000, 180000000, 75000000, 127000000, 250000000]
[5.1, 8.0, 5.

20.4
15000000
[262000000, 160000000, 150000000, 200000000, 175000000, 35000000, 180000000, 75000000, 127000000, 250000000, 300000000, 150000000, 175000000, 80000000, 5000000, 125000000, 230000000, 90000000, 185000000, 25000000, 175000000, 152000000, 9000000, 217000000, 20000000, 20000000, 28000000, 55000000, 34000000, 200000000, 75000000, 15000000]
[5.1, 8.0, 5.5, 4.3, 5.0, 20.0, 4.7, 13.8, 4.8, 4.9, 2.2, 3.5, 4.6, 3.9, 50.5, 4.2, 3.5, 10.7, 3.0, 9.3, 2.2, 3.2, 31.0, 2.8, 15.6, 22.1, 5.0, 6.9, 6.7, 5.3, 8.4, 20.4]
2.5
70000000
[262000000, 160000000, 150000000, 200000000, 175000000, 35000000, 180000000, 75000000, 127000000, 250000000, 300000000, 150000000, 175000000, 80000000, 5000000, 125000000, 230000000, 90000000, 185000000, 25000000, 175000000, 152000000, 9000000, 217000000, 20000000, 20000000, 28000000, 55000000, 34000000, 200000000, 75000000, 15000000, 70000000]
[5.1, 8.0, 5.5, 4.3, 5.0, 20.0, 4.7, 13.8, 4.8, 4.9, 2.2, 3.5, 4.6, 3.9, 50.5, 4.2, 3.5, 10.7, 3.0, 9.3, 2.2, 3.2, 31.0,

4.1
45000000
[262000000, 160000000, 150000000, 200000000, 175000000, 35000000, 180000000, 75000000, 127000000, 250000000, 300000000, 150000000, 175000000, 80000000, 5000000, 125000000, 230000000, 90000000, 185000000, 25000000, 175000000, 152000000, 9000000, 217000000, 20000000, 20000000, 28000000, 55000000, 34000000, 200000000, 75000000, 15000000, 70000000, 55000000, 104000000, 40000000, 185000000, 50000000, 120000000, 195000000, 30000000, 97000000, 38000000, 28000000, 25000000, 45000000]
[5.1, 8.0, 5.5, 4.3, 5.0, 20.0, 4.7, 13.8, 4.8, 4.9, 2.2, 3.5, 4.6, 3.9, 50.5, 4.2, 3.5, 10.7, 3.0, 9.3, 2.2, 3.2, 31.0, 2.8, 15.6, 22.1, 5.0, 6.9, 6.7, 5.3, 8.4, 20.4, 2.5, 6.4, 3.9, 4.3, 1.4, 4.3, 1.2, 2.1, 6.1, 2.5, 3.3, 4.6, 8.1, 4.1]
1.8
70000000
[262000000, 160000000, 150000000, 200000000, 175000000, 35000000, 180000000, 75000000, 127000000, 250000000, 300000000, 150000000, 175000000, 80000000, 5000000, 125000000, 230000000, 90000000, 185000000, 25000000, 175000000, 152000000, 9000000, 217000000

12.4
12000000
[262000000, 160000000, 150000000, 200000000, 175000000, 35000000, 180000000, 75000000, 127000000, 250000000, 300000000, 150000000, 175000000, 80000000, 5000000, 125000000, 230000000, 90000000, 185000000, 25000000, 175000000, 152000000, 9000000, 217000000, 20000000, 20000000, 28000000, 55000000, 34000000, 200000000, 75000000, 15000000, 70000000, 55000000, 104000000, 40000000, 185000000, 50000000, 120000000, 195000000, 30000000, 97000000, 38000000, 28000000, 25000000, 45000000, 70000000, 60000000, 20000000, 5000000, 111000000, 30000000, 50000000, 66000000, 84000000, 20000000, 12000000]
[5.1, 8.0, 5.5, 4.3, 5.0, 20.0, 4.7, 13.8, 4.8, 4.9, 2.2, 3.5, 4.6, 3.9, 50.5, 4.2, 3.5, 10.7, 3.0, 9.3, 2.2, 3.2, 31.0, 2.8, 15.6, 22.1, 5.0, 6.9, 6.7, 5.3, 8.4, 20.4, 2.5, 6.4, 3.9, 4.3, 1.4, 4.3, 1.2, 2.1, 6.1, 2.5, 3.3, 4.6, 8.1, 4.1, 1.8, 2.9, 4.8, 25.0, 2.6, 3.3, 2.7, 1.7, 5.2, 2.4, 12.4]
1.4
42000000
[262000000, 160000000, 150000000, 200000000, 175000000, 35000000, 180000000, 75000000,

11.4
5000000
[262000000, 160000000, 150000000, 200000000, 175000000, 35000000, 180000000, 75000000, 127000000, 250000000, 300000000, 150000000, 175000000, 80000000, 5000000, 125000000, 230000000, 90000000, 185000000, 25000000, 175000000, 152000000, 9000000, 217000000, 20000000, 20000000, 28000000, 55000000, 34000000, 200000000, 75000000, 15000000, 70000000, 55000000, 104000000, 40000000, 185000000, 50000000, 120000000, 195000000, 30000000, 97000000, 38000000, 28000000, 25000000, 45000000, 70000000, 60000000, 20000000, 5000000, 111000000, 30000000, 50000000, 66000000, 84000000, 20000000, 12000000, 42000000, 150000000, 60000000, 24000000, 45000000, 85000000, 110000000, 5300000, 5000000]
[5.1, 8.0, 5.5, 4.3, 5.0, 20.0, 4.7, 13.8, 4.8, 4.9, 2.2, 3.5, 4.6, 3.9, 50.5, 4.2, 3.5, 10.7, 3.0, 9.3, 2.2, 3.2, 31.0, 2.8, 15.6, 22.1, 5.0, 6.9, 6.7, 5.3, 8.4, 20.4, 2.5, 6.4, 3.9, 4.3, 1.4, 4.3, 1.2, 2.1, 6.1, 2.5, 3.3, 4.6, 8.1, 4.1, 1.8, 2.9, 4.8, 25.0, 2.6, 3.3, 2.7, 1.7, 5.2, 2.4, 12.4, 1.4, 2.2, 

...

[262000000, 160000000, 150000000, 200000000, 175000000, 35000000, 180000000, 75000000, 127000000, 250000000, 300000000, 150000000, 175000000, 80000000, 5000000, 125000000, 230000000, 90000000, 185000000, 25000000, 175000000, 152000000, 9000000, 217000000, 20000000, 20000000, 28000000, 55000000, 34000000, 200000000, 75000000, 15000000, 70000000, 55000000, 104000000, 40000000, 185000000, 50000000, 120000000, 195000000, 30000000, 97000000, 38000000, 28000000, 25000000, 45000000, 70000000, 60000000, 20000000, 5000000, 111000000, 30000000, 50000000, 66000000, 84000000, 20000000, 12000000, 42000000, 150000000, 60000000, 24000000, 45000000, 85000000, 110000000, 5300000, 5000000, 110000000, 180000000, 20000000, 175000000, 150000000, 10000000, 63000000, 35000000, 10000000, 11000000, 100000000, 125000000, 25000000, 10000000, 10000000, 40000000, 21000000, 35000000, 'nf', 58000000, 38000000, 40000000, 24000000, 25000000, 29000000, 15000000, 40000000, 40000000, 7000000, 12000000, 7400000, 21000000,

[262000000, 160000000, 150000000, 200000000, 175000000, 35000000, 180000000, 75000000, 127000000, 250000000, 300000000, 150000000, 175000000, 80000000, 5000000, 125000000, 230000000, 90000000, 185000000, 25000000, 175000000, 152000000, 9000000, 217000000, 20000000, 20000000, 28000000, 55000000, 34000000, 200000000, 75000000, 15000000, 70000000, 55000000, 104000000, 40000000, 185000000, 50000000, 120000000, 195000000, 30000000, 97000000, 38000000, 28000000, 25000000, 45000000, 70000000, 60000000, 20000000, 5000000, 111000000, 30000000, 50000000, 66000000, 84000000, 20000000, 12000000, 42000000, 150000000, 60000000, 24000000, 45000000, 85000000, 110000000, 5300000, 5000000, 110000000, 180000000, 20000000, 175000000, 150000000, 10000000, 63000000, 35000000, 10000000, 11000000, 100000000, 125000000, 25000000, 10000000, 10000000, 40000000, 21000000, 35000000, 'nf', 58000000, 38000000, 40000000, 24000000, 25000000, 29000000, 15000000, 40000000, 40000000, 7000000, 12000000, 7400000, 21000000,

[262000000, 160000000, 150000000, 200000000, 175000000, 35000000, 180000000, 75000000, 127000000, 250000000, 300000000, 150000000, 175000000, 80000000, 5000000, 125000000, 230000000, 90000000, 185000000, 25000000, 175000000, 152000000, 9000000, 217000000, 20000000, 20000000, 28000000, 55000000, 34000000, 200000000, 75000000, 15000000, 70000000, 55000000, 104000000, 40000000, 185000000, 50000000, 120000000, 195000000, 30000000, 97000000, 38000000, 28000000, 25000000, 45000000, 70000000, 60000000, 20000000, 5000000, 111000000, 30000000, 50000000, 66000000, 84000000, 20000000, 12000000, 42000000, 150000000, 60000000, 24000000, 45000000, 85000000, 110000000, 5300000, 5000000, 110000000, 180000000, 20000000, 175000000, 150000000, 10000000, 63000000, 35000000, 10000000, 11000000, 100000000, 125000000, 25000000, 10000000, 10000000, 40000000, 21000000, 35000000, 'nf', 58000000, 38000000, 40000000, 24000000, 25000000, 29000000, 15000000, 40000000, 40000000, 7000000, 12000000, 7400000, 21000000,

0.1
250000
[262000000, 160000000, 150000000, 200000000, 175000000, 35000000, 180000000, 75000000, 127000000, 250000000, 300000000, 150000000, 175000000, 80000000, 5000000, 125000000, 230000000, 90000000, 185000000, 25000000, 175000000, 152000000, 9000000, 217000000, 20000000, 20000000, 28000000, 55000000, 34000000, 200000000, 75000000, 15000000, 70000000, 55000000, 104000000, 40000000, 185000000, 50000000, 120000000, 195000000, 30000000, 97000000, 38000000, 28000000, 25000000, 45000000, 70000000, 60000000, 20000000, 5000000, 111000000, 30000000, 50000000, 66000000, 84000000, 20000000, 12000000, 42000000, 150000000, 60000000, 24000000, 45000000, 85000000, 110000000, 5300000, 5000000, 110000000, 180000000, 20000000, 175000000, 150000000, 10000000, 63000000, 35000000, 10000000, 11000000, 100000000, 125000000, 25000000, 10000000, 10000000, 40000000, 21000000, 35000000, 'nf', 58000000, 38000000, 40000000, 24000000, 25000000, 29000000, 15000000, 40000000, 40000000, 7000000, 12000000, 7400000

[262000000, 160000000, 150000000, 200000000, 175000000, 35000000, 180000000, 75000000, 127000000, 250000000, 300000000, 150000000, 175000000, 80000000, 5000000, 125000000, 230000000, 90000000, 185000000, 25000000, 175000000, 152000000, 9000000, 217000000, 20000000, 20000000, 28000000, 55000000, 34000000, 200000000, 75000000, 15000000, 70000000, 55000000, 104000000, 40000000, 185000000, 50000000, 120000000, 195000000, 30000000, 97000000, 38000000, 28000000, 25000000, 45000000, 70000000, 60000000, 20000000, 5000000, 111000000, 30000000, 50000000, 66000000, 84000000, 20000000, 12000000, 42000000, 150000000, 60000000, 24000000, 45000000, 85000000, 110000000, 5300000, 5000000, 110000000, 180000000, 20000000, 175000000, 150000000, 10000000, 63000000, 35000000, 10000000, 11000000, 100000000, 125000000, 25000000, 10000000, 10000000, 40000000, 21000000, 35000000, 'nf', 58000000, 38000000, 40000000, 24000000, 25000000, 29000000, 15000000, 40000000, 40000000, 7000000, 12000000, 7400000, 21000000,

In [17]:
 budget_2017_df[budget_2017_df['Production_Budget'] == 'nf']

,Production_Budget
84,nf
113,nf
122,nf
123,nf
142,nf
...,...
728,nf
729,nf
730,nf
731,nf


In [18]:
# Extract theater count and average run for all 2017 movies

all_movies_theater_count_list = []
all_movies_avg_run_list = []

for i in range(row_count):
    r = requests.get(df.loc[i,'summary_page_url']) #location in my dataframe
    s = bs(r.text)
    ex_all_tables = s.find_all('table')
    count = i

    for table in ex_all_tables:
        all_rows = table.find_all('tr')
        found_theater = False
        
        for row in all_rows:
            if ('Theater' in row.text and 'counts' in row.text):
                found_theater = True

                ex_row_theater_count = row
                print(row.text)
                ex_strg_theater_count = row.text

                pattern1 = re.compile(r"[-+]?(?:\d*\.\d+|\d+)")
                pattern2 = re.compile(r"[-+]?(?:\d*\,\d+|\d+)")
                print(pattern1.findall(ex_strg_theater_count))
                print(pattern2.findall(ex_strg_theater_count))
                
                theater_count_list1 = pattern1.findall(ex_strg_theater_count)
                theater_count_list2 = pattern2.findall(ex_strg_theater_count)
 
                theater_run = float(theater_count_list1[-1])
                print(theater_run)
                theater_count = int(theater_count_list2[0].replace(',', ''))
                print(theater_run)

                all_movies_theater_count_list.append(theater_count)
                all_movies_avg_run_list.append(theater_run)
                break
           
        if(found_theater):
            break     
                
    if(not found_theater):
        theater_count = 'nf'
        theater_run = 'nf'

        all_movies_theater_count_list.append(theater_count)
        all_movies_avg_run_list.append(theater_run)
            
            
theater_count_2017_df = pd.DataFrame(all_movies_theater_count_list,
                                     columns = ['Theater_Count'])
avg_run_2017_df = pd.DataFrame(all_movies_avg_run_list,
                               columns = ['Average_Run'])
       

theater_count_2017_df.to_pickle('theater_count_2017_pk')
avg_run_2017_df.to_pickle('avg_run_2017_pk')

Theater counts:4,232 opening theaters/4,232 max. theaters, 6.7 weeks average run per theater
['4', '232', '4', '232', '6.7']
['4,232', '4,232', '6', '7']
6.7
6.7
Theater counts:4,210 opening theaters/4,210 max. theaters, 8.5 weeks average run per theater
['4', '210', '4', '210', '8.5']
['4,210', '4,210', '8', '5']
8.5
8.5
Theater counts:4,165 opening theaters/4,165 max. theaters, 9.3 weeks average run per theater
['4', '165', '4', '165', '9.3']
['4,165', '4,165', '9', '3']
9.3
9.3
Theater counts:4,347 opening theaters/4,347 max. theaters, 7.0 weeks average run per theater
['4', '347', '4', '347', '7.0']
['4,347', '4,347', '7', '0']
7.0
7.0
Theater counts:4,348 opening theaters/4,348 max. theaters, 8.0 weeks average run per theater
['4', '348', '4', '348', '8.0']
['4,348', '4,348', '8', '0']
8.0
8.0
Theater counts:4,103 opening theaters/4,148 max. theaters, 7.4 weeks average run per theater
['4', '103', '4', '148', '7.4']
['4,103', '4,148', '7', '4']
7.4
7.4
Theater counts:4,080 opening

Theater counts:3,304 opening theaters/3,326 max. theaters, 3.9 weeks average run per theater
['3', '304', '3', '326', '3.9']
['3,304', '3,326', '3', '9']
3.9
3.9
Theater counts:3,024 opening theaters/3,098 max. theaters, 4.8 weeks average run per theater
['3', '024', '3', '098', '4.8']
['3,024', '3,098', '4', '8']
4.8
4.8
Theater counts:3,451 opening theaters/3,451 max. theaters, 5.0 weeks average run per theater
['3', '451', '3', '451', '5.0']
['3,451', '3,451', '5', '0']
5.0
5.0
Theater counts:3,006 opening theaters/3,342 max. theaters, 9.6 weeks average run per theater
['3', '006', '3', '342', '9.6']
['3,006', '3,342', '9', '6']
9.6
9.6
Theater counts:2,388 opening theaters/2,388 max. theaters, 4.3 weeks average run per theater
['2', '388', '2', '388', '4.3']
['2,388', '2,388', '4', '3']
4.3
4.3
Theater counts:4 opening theaters/1,802 max. theaters, 7.4 weeks average run per theater
['4', '1', '802', '7.4']
['4', '1,802', '7', '4']
7.4
7.4
Theater counts:3,501 opening theaters/3,511

Theater counts:3,163 opening theaters/3,163 max. theaters, 3.0 weeks average run per theater
['3', '163', '3', '163', '3.0']
['3,163', '3,163', '3', '0']
3.0
3.0
Theater counts:4 opening theaters/1,213 max. theaters, 8.7 weeks average run per theater
['4', '1', '213', '8.7']
['4', '1,213', '8', '7']
8.7
8.7
Theater counts:419 opening theaters/425 max. theaters, 2.8 weeks average run per theater
['419', '425', '2.8']
['419', '425', '2', '8']
2.8
2.8
Theater counts:2,464 opening theaters/2,464 max. theaters, 3.0 weeks average run per theater
['2', '464', '2', '464', '3.0']
['2,464', '2,464', '3', '0']
3.0
3.0
Theater counts:2,970 opening theaters/2,996 max. theaters, 3.5 weeks average run per theater
['2', '970', '2', '996', '3.5']
['2,970', '2,996', '3', '5']
3.5
3.5
Theater counts:2,577 opening theaters/2,577 max. theaters, 3.7 weeks average run per theater
['2', '577', '2', '577', '3.7']
['2,577', '2,577', '3', '7']
3.7
3.7
Theater counts:4 opening theaters/1,733 max. theaters, 7.7 we

Theater counts:2,161 opening theaters/2,161 max. theaters, 1.9 weeks average run per theater
['2', '161', '2', '161', '1.9']
['2,161', '2,161', '1', '9']
1.9
1.9
Theater counts:846 opening theaters/846 max. theaters, 4.6 weeks average run per theater
['846', '846', '4.6']
['846', '846', '4', '6']
4.6
4.6
Theater counts:2,046 opening theaters/2,046 max. theaters, 2.1 weeks average run per theater
['2', '046', '2', '046', '2.1']
['2,046', '2,046', '2', '1']
2.1
2.1
Theater counts:4 opening theaters/214 max. theaters, 4.1 weeks average run per theater
['4', '214', '4.1']
['4', '214', '4', '1']
4.1
4.1
Theater counts:626 opening theaters/720 max. theaters, 3.8 weeks average run per theater
['626', '720', '3.8']
['626', '720', '3', '8']
3.8
3.8
Theater counts:4 opening theaters/650 max. theaters, 3.2 weeks average run per theater
['4', '650', '3.2']
['4', '650', '3', '2']
3.2
3.2
Theater counts:271 opening theaters/1,708 max. theaters, 4.1 weeks average run per theater
['271', '1', '708', '

Theater counts:4 opening theaters/380 max. theaters, 2.7 weeks average run per theater
['4', '380', '2.7']
['4', '380', '2', '7']
2.7
2.7
Theater counts:9 opening theaters/2,851 max. theaters, 5.6 weeks average run per theater
['9', '2', '851', '5.6']
['9', '2,851', '5', '6']
5.6
5.6
Theater counts:9 opening theaters/262 max. theaters, 3.5 weeks average run per theater
['9', '262', '3.5']
['9', '262', '3', '5']
3.5
3.5
Theater counts:3 opening theaters/126 max. theaters, 5.6 weeks average run per theater
['3', '126', '5.6']
['3', '126', '5', '6']
5.6
5.6
Theater counts:502 opening theaters/502 max. theaters, 1.0 weeks average run per theater
['502', '502', '1.0']
['502', '502', '1', '0']
1.0
1.0
Theater counts:848 opening theaters/848 max. theaters, 1.8 weeks average run per theater
['848', '848', '1.8']
['848', '848', '1', '8']
1.8
1.8
Theater counts:173 opening theaters/173 max. theaters, 2.3 weeks average run per theater
['173', '173', '2.3']
['173', '173', '2', '3']
2.3
2.3
Theater

...

Theater counts:1 opening theaters/8 max. theaters, 4.3 weeks average run per theater
['1', '8', '4.3']
['1', '8', '4', '3']
4.3
4.3
Theater counts:2 opening theaters/10 max. theaters, 5.4 weeks average run per theater
['2', '10', '5.4']
['2', '10', '5', '4']
5.4
5.4
Theater counts:2 opening theaters/6 max. theaters, 10.8 weeks average run per theater
['2', '6', '10.8']
['2', '6', '10', '8']
10.8
10.8
Theater counts:18 opening theaters/18 max. theaters, 2.1 weeks average run per theater
['18', '18', '2.1']
['18', '18', '2', '1']
2.1
2.1
Theater counts:2 opening theaters/12 max. theaters, 6.2 weeks average run per theater
['2', '12', '6.2']
['2', '12', '6', '2']
6.2
6.2
Theater counts:20 opening theaters/20 max. theaters, 1.4 weeks average run per theater
['20', '20', '1.4']
['20', '20', '1', '4']
1.4
1.4
Theater counts:2 opening theaters/13 max. theaters, 4.2 weeks average run per theater
['2', '13', '4.2']
['2', '13', '4', '2']
4.2
4.2
Theater counts:2 opening theaters/32 max. theaters

Theater counts:1 opening theaters/11 max. theaters, 4.0 weeks average run per theater
['1', '11', '4.0']
['1', '11', '4', '0']
4.0
4.0
Theater counts:2 opening theaters/22 max. theaters, 1.8 weeks average run per theater
['2', '22', '1.8']
['2', '22', '1', '8']
1.8
1.8
Theater counts:4 opening theaters/107 max. theaters, 5.6 weeks average run per theater
['4', '107', '5.6']
['4', '107', '5', '6']
5.6
5.6
Theater counts:1 opening theaters/3 max. theaters, 3.0 weeks average run per theater
['1', '3', '3.0']
['1', '3', '3', '0']
3.0
3.0
Theater counts:4 opening theaters/97 max. theaters, 1.2 weeks average run per theater
['4', '97', '1.2']
['4', '97', '1', '2']
1.2
1.2
Theater counts:14 opening theaters/14 max. theaters, 1.4 weeks average run per theater
['14', '14', '1.4']
['14', '14', '1', '4']
1.4
1.4
Theater counts:2 opening theaters/4 max. theaters, 5.8 weeks average run per theater
['2', '4', '5.8']
['2', '4', '5', '8']
5.8
5.8
Theater counts:3 opening theaters/39 max. theaters, 0.4

Theater counts:26 opening theaters/26 max. theaters, 1.2 weeks average run per theater
['26', '26', '1.2']
['26', '26', '1', '2']
1.2
1.2
Theater counts:11 opening theaters/11 max. theaters, 1.4 weeks average run per theater
['11', '11', '1.4']
['11', '11', '1', '4']
1.4
1.4
Theater counts:25 opening theaters/25 max. theaters, 1.2 weeks average run per theater
['25', '25', '1.2']
['25', '25', '1', '2']
1.2
1.2
Theater counts:2 opening theaters/2 max. theaters, 5.0 weeks average run per theater
['2', '2', '5.0']
['2', '2', '5', '0']
5.0
5.0
Theater counts:6 opening theaters/6 max. theaters, 1.2 weeks average run per theater
['6', '6', '1.2']
['6', '6', '1', '2']
1.2
1.2
Theater counts:1 opening theaters/2 max. theaters, 5.0 weeks average run per theater
['1', '2', '5.0']
['1', '2', '5', '0']
5.0
5.0
Theater counts:2 opening theaters/2 max. theaters, 1.5 weeks average run per theater
['2', '2', '1.5']
['2', '2', '1', '5']
1.5
1.5
Theater counts:1 opening theaters/3 max. theaters, 2.7 wee

Theater counts:1 opening theaters/4 max. theaters, 2.8 weeks average run per theater
['1', '4', '2.8']
['1', '4', '2', '8']
2.8
2.8
Theater counts:77 opening theaters/77 max. theaters, 1.2 weeks average run per theater
['77', '77', '1.2']
['77', '77', '1', '2']
1.2
1.2
Theater counts:3 opening theaters/3 max. theaters, 4.3 weeks average run per theater
['3', '3', '4.3']
['3', '3', '4', '3']
4.3
4.3
Theater counts:1 opening theaters/1 max. theaters, 1.0 weeks average run per theater
['1', '1', '1.0']
['1', '1', '1', '0']
1.0
1.0
Theater counts:1 opening theaters/1 max. theaters, 3.0 weeks average run per theater
['1', '1', '3.0']
['1', '1', '3', '0']
3.0
3.0
Theater counts:4 opening theaters/8 max. theaters, 1.5 weeks average run per theater
['4', '8', '1.5']
['4', '8', '1', '5']
1.5
1.5
Theater counts:2 opening theaters/2 max. theaters, 1.0 weeks average run per theater
['2', '2', '1.0']
['2', '2', '1', '0']
1.0
1.0
Theater counts:1 opening theaters/2 max. theaters, 3.5 weeks average r

Theater counts:2 opening theaters/2 max. theaters, 1.5 weeks average run per theater
['2', '2', '1.5']
['2', '2', '1', '5']
1.5
1.5
Theater counts:2 opening theaters/3 max. theaters, 7.3 weeks average run per theater
['2', '3', '7.3']
['2', '3', '7', '3']
7.3
7.3
Theater counts:1 opening theaters/1 max. theaters, 2.0 weeks average run per theater
['1', '1', '2.0']
['1', '1', '2', '0']
2.0
2.0
Theater counts:1 opening theaters/1 max. theaters, 2.0 weeks average run per theater
['1', '1', '2.0']
['1', '1', '2', '0']
2.0
2.0
Theater counts:2 opening theaters/2 max. theaters, 1.0 weeks average run per theater
['2', '2', '1.0']
['2', '2', '1', '0']
1.0
1.0
Theater counts:1 opening theaters/3 max. theaters, 2.3 weeks average run per theater
['1', '3', '2.3']
['1', '3', '2', '3']
2.3
2.3
Theater counts:2 opening theaters/2 max. theaters, 1.0 weeks average run per theater
['2', '2', '1.0']
['2', '2', '1', '0']
1.0
1.0
Theater counts:2,058 opening theaters/2,058 max. theaters, 2.2 weeks average

In [19]:
len(all_movies_theater_count_list)
avg_run_2017_df

,Average_Run
0,6.7
1,8.5
2,9.3
3,7.0
4,8.0
...,...
729,7.5
730,nf
731,1.6
732,4.2


In [20]:
theater_count_2017_df

,Theater_Count
0,4232
1,4210
2,4165
3,4347
4,4348
...,...
729,2
730,nf
731,1
732,2


In [21]:
# Extract source for all 2017 movies

all_movies_source_list = []

for i in range(row_count):
    summary_url = df.loc[i,'summary_page_url']
    print(summary_url)
    
    r = requests.get(summary_url) #location in my dataframe
    s = bs(r.text)
    ex_all_tables = s.find_all('table')
    
    for table in ex_all_tables:
        all_rows = table.find_all('tr')
        found_source = False
        for row in all_rows:
    #         print(row)
            if ('Source' in row.text):
                found_source = True
                source_strg = row.find_all('a')[0].text
                print(source_strg)
                all_movies_source_list.append(source_strg)
                break
        
        if (found_source):
            break
    if (not found_source):
        source_strg = 'nf'
        print()
        all_movies_source_list.append(source_strg)
    

source_2017_df = pd.DataFrame(all_movies_source_list, columns = ['Source'])        
source_2017_df.to_pickle('source_2017_pk') 
    

https://www.the-numbers.com/movie/Star-Wars-Ep-VIII-The-Last-Jedi#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Beauty-and-the-Beast-(2017)#tab=summary
Remake
https://www.the-numbers.com/movie/Wonder-Woman-(2017)#tab=summary
Based on Comic/Graphic Novel
https://www.the-numbers.com/movie/Guardians-of-the-Galaxy-Vol-2#tab=summary
Based on Comic/Graphic Novel
https://www.the-numbers.com/movie/Spider-Man-Homecoming#tab=summary
Based on Comic/Graphic Novel
https://www.the-numbers.com/movie/It-(2017)#tab=summary
Remake
https://www.the-numbers.com/movie/Thor-Ragnarok#tab=summary
Based on Comic/Graphic Novel
https://www.the-numbers.com/movie/Despicable-Me-3#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Logan-(2017)#tab=summary
Based on Comic/Graphic Novel
https://www.the-numbers.com/movie/Fate-of-the-Furious-The#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Justice-League-(2017)#tab=summary
Based on Comic/Graphic Novel
https://www.the

Original Screenplay
https://www.the-numbers.com/movie/Gifted#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Three-Billboards-Outside-Ebbing-Missouri#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Bye-Bye-Man-The#tab=summary
Based on Fiction Book/Short Story
https://www.the-numbers.com/movie/Victoria-and-Abdul#tab=summary
Based on Factual Book/Article
https://www.the-numbers.com/movie/Rough-Night#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/My-Little-Pony-The-Movie-(2017)#tab=summary
Based on Toy
https://www.the-numbers.com/movie/Ballerina-(Canada)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Sleepless-(2016)#tab=summary
Based on Movie
https://www.the-numbers.com/movie/Diary-of-a-Wimpy-Kid-The-Long-Haul#tab=summary
Based on Fiction Book/Short Story
https://www.the-numbers.com/movie/Circle-The-(2016)#tab=summary
Based on Fiction Book/Short Story
https://www.the-numbers.com/movie/Manchester-by-the-Sea#tab=summ

Original Screenplay
https://www.the-numbers.com/movie/Ingrid-Goes-West#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/2017-Oscar-Shorts#tab=summary
Compilation
https://www.the-numbers.com/movie/Kedi-(Turkey)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Zhan-Lang-2-(China)-(2017)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/My-Cousin-Rachel#tab=summary
Remake
https://www.the-numbers.com/movie/Mayweather-vs-McGregor#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Hacksaw-Ridge#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Question-of-Faith-A-(2017)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Is-Genesis-History#tab=summary
Based on Religious Text
https://www.the-numbers.com/movie/Hazlo-Como-Hombre-(Chile)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Doctor-Strange-(2016)#tab=summary
Based on Comic/Graphic Novel
https://www.the-numb

Based on Web Series
https://www.the-numbers.com/movie/Only-Living-Boy-in-New-York-The-(2017)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Visages-villages-(France)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Sage-femme-(France)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Jiyi-dashi-(China)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Master-(South-Korea)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Tommys-Honour#tab=summary
Based on Factual Book/Article
https://www.the-numbers.com/movie/Novitiate#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Bitter-Harvest#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Dolores-(Documentary)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Human-Flow-(Germany)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Phantom-Thread#tab=summary
Original Screenplay
https://ww

Based on Real Life Events
https://www.the-numbers.com/movie/9-11#tab=summary
Based on Play
https://www.the-numbers.com/movie/David-Lynch-The-Art-Life#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Valley-of-Bones#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Polina-danser-sa-vie-(France)#tab=summary
Based on Fiction Book/Short Story
https://www.the-numbers.com/movie/Journey-The-(UK-2016)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Manifesto-(Germany)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Bombshell-The-Hedy-Lamarr-Story#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Three-Generations#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Elser-(Germany)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Ex-Libris-The-New-York-Public-Library#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Void-The-(Canada)#ta

Based on Real Life Events
https://www.the-numbers.com/movie/God-of-War-(China)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Paris-Opera-The-(Switzerland)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Gottliche-Ordnung-Die-(Switzerland)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Voyage-a-travers-le-cinema-francais-(France)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/tout-nouveau-testament-Le-(Belgium)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Mei-Hao-De-Yi-Wai-(China)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Ornitologo-O-(Portugal)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Keep-Quiet-(UK)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Mal-de-Pierres-(France)#tab=summary
Based on Fiction Book/Short Story
https://www.the-numbers.com/movie/Person-to-Person#tab=summary
Original Screenplay
https://w

Original Screenplay
https://www.the-numbers.com/movie/RV-Resurrected-Victims-(South-Korea)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Better-Watch-Out-(Australia)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Slava-(Bulgaria)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Carrie-Pilby#tab=summary
Based on Fiction Book/Short Story
https://www.the-numbers.com/movie/Leon-Morin-Pretre#tab=summary
Based on Fiction Book/Short Story
https://www.the-numbers.com/movie/Black-Code-(Canada)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Man-From-Outer-Space-The-(2017)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Bobbi-Jene#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/All-the-Rage-Saved-by-Sarno#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Burden-(Documentary)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Eshtebak

Based on Real Life Events
https://www.the-numbers.com/movie/Jazz-Funeral-The#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Miss-Sloane#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Spices-of-Liberty#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Harry-Benson-Shoot-First#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Sur-quel-pied-danser-(France)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Pathological-Optimist-The-(Documentary)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Last-Rampage-(2017)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Chosen-(2016)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Fuzz-The-(2017)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Freedom-to-Marry-The#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Offret#tab=summary
Original Screenpla

Based on Real Life Events
https://www.the-numbers.com/movie/Amityville-The-Awakening#tab=summary
Based on Fiction Book/Short Story
https://www.the-numbers.com/movie/This-Last-Lonely-Place#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Love-Sweat-and-Tears#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Helsings-The#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Seed-The-Untold-Story#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/2-22-(2017)#tab=summary
Based on Fiction Book/Short Story
https://www.the-numbers.com/movie/Mifune#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Gridiron-Heroes#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/National-Bird#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Nightlights-(2014)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Muchenik-(Russia)#tab=summary
Original Screenplay


In [29]:
len(all_movies_source_list)
# source_2017_df[source_2017_df['Source'] == 'nf']

734

In [30]:
all_movies_creative_type_list = []

for i in range(row_count):
    summary_url = df.loc[i,'summary_page_url']
    
    r = requests.get(summary_url) #location in my dataframe
    s = bs(r.text)
    ex_all_tables = s.find_all('table')
    
    for table in ex_all_tables:
        all_rows = table.find_all('tr')
        found_ctype = False
        for row in all_rows:
#             print(row.text)
            if ('Creative' in row.text and 'Type' in row.text):
                found_ctype = True
#                 print(row)
                ctype = row.find_all('a')[0].text
                print(ctype)
                
                all_movies_creative_type_list.append(ctype)
                break
        
        if (found_ctype):
            break
    if (not found_ctype):
        ctype = 'nf'
        print('df')
        all_movies_creative_type_list.append(ctype)
    

creative_type_2017_df = pd.DataFrame(all_movies_creative_type_list,
                                     columns = ['Creative_Type'])        
creative_type_2017_df.to_pickle('creative_type_2017_pk') 

Science Fiction
Fantasy
Super Hero
Super Hero
Super Hero
Historical Fiction
Super Hero
Kids Fiction
Super Hero
Contemporary Fiction
Super Hero
Dramatization
Kids Fiction
Kids Fiction
Contemporary Fiction
Kids Fiction
Historical Fiction
Science Fiction
Science Fiction
Dramatization
Kids Fiction
Science Fiction
Contemporary Fiction
Science Fiction
Dramatization
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Science Fiction
Kids Fiction
Contemporary Fiction
Contemporary Fiction
Historical Fiction
Contemporary Fiction
Contemporary Fiction
Science Fiction
Kids Fiction
Kids Fiction
Historical Fiction
Contemporary Fiction
Science Fiction
Kids Fiction
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Kids Fiction
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Kids Fiction
Historical Fiction
Dramatization
Science Fiction
Dramatization
Contemporary Fiction
Dramatization
Contemporary Fiction
Historical Fiction
Kids Fiction
Contempo

Factual
Dramatization
Contemporary Fiction
Historical Fiction
Contemporary Fiction
Factual
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Factual
Factual
Factual
Contemporary Fiction
Factual
Factual
Contemporary Fiction
Contemporary Fiction
Factual
Contemporary Fiction
Factual
Factual
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Factual
Contemporary Fiction
Factual
Contemporary Fiction
Contemporary Fiction
Factual
Factual
Factual
Contemporary Fiction
Science Fiction
Factual
Factual
Contemporary Fiction
Factual
Science Fiction
Factual
Contemporary Fiction
Contemporary Fiction
Factual
Factual
Fantasy
Dramatization
Factual
Contemporary Fiction
Contemporary Fiction
Science Fiction
Factual
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Historical Fiction
Factual
Science Fiction
Factual
Factual
Factual
Contemporary Fiction
Factual
Factual


In [ ]:
len(all_movies_creative_type_list)
creative_type_2017_df

In [ ]:
creative_type_2017_df[creative_type_2017_df['Creative_Type'] == 'nf']